<a href="https://colab.research.google.com/github/zsajadifar/P-D-Information-Systems-and-Signal-Processing/blob/main/dilated_convolutional_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Load required libraries
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, Activation
from keras import regularizers
import tensorflow as tf 
from tensorflow.python.ops.linalg_ops import norm
import h5py
import numpy as np
import matplotlib.pyplot as plt


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
time_window=640
layers=3
kernel_size=3
filters_spatial=8
filters_dilated=16

eeg = tf.keras.layers.Input(shape=[time_window, 64])
env1 = tf.keras.layers.Input(shape=[time_window, 1])
env2 = tf.keras.layers.Input(shape=[time_window, 1])

#add model layers
## ---- add your code ----here

# Activations to apply
activations = ["relu"] * layers

# Spatial convolution
env1_conv = env1
env2_conv = env2
eeg_conv = tf.keras.layers.Conv1D(filters_spatial, kernel_size=1)(eeg)

# Dilated convolution
for l in range(layers):
  # eeg
  eeg_conv  = tf.keras.layers.Conv1D(filters_dilated, kernel_size=kernel_size, dilation_rate=kernel_size ** l,
                                            strides=1, activation=activations[l])(eeg_conv)

  # env 
  env1_conv = tf.keras.layers.Conv1D(filters_dilated, kernel_size=kernel_size, dilation_rate=kernel_size ** l,
                                            strides=1, activation=activations[l])(env1_conv)                                       

  env2_conv = tf.keras.layers.Conv1D(filters_dilated, kernel_size=kernel_size, dilation_rate=kernel_size ** l,
                                            strides=1, activation=activations[l])(env2_conv) 



# Classification
cos1 = tf.keras.layers.Dot(1, normalize=True)([eeg_conv, env1_conv])
cos2 = tf.keras.layers.Dot(1, normalize=True)([eeg_conv, env2_conv])
cos_similarity = tf.keras.layers.Concatenate()([cos1, cos2])
cos_flat = tf.keras.layers.Flatten()(cos_similarity)
out1 = tf.keras.layers.Dense(1, activation="sigmoid")(cos_flat)

# 1 output per batch
#out = tf.keras.layers.Reshape([1], name=output_name)(out1)
model = tf.keras.Model(inputs=[eeg, env1, env2], outputs=[out1])


In [4]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 640, 64)]    0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 640, 8)       520         ['input_1[0][0]']                
                                                                                                  
 input_2 (InputLayer)           [(None, 640, 1)]     0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 640, 1)]     0           []                               
                                                                                              

In [5]:
def batch_equalizer(eeg, env1, env2, labels):
    # present each of the eeg segments twice, where the envelopes, and thus the labels 
    # are swapped around. EEG presented in small segments [bs, window_length, 64]
    return np.concatenate([eeg,eeg], axis=0), np.concatenate([env1, env2], axis=0),np.concatenate([env2, env1], axis=0), np.concatenate([labels, (labels+1)%2], axis=0)


In [6]:
eeg_path = '/content/drive/MyDrive/Colab Notebooks/P&D_phase2/data/modified data/eeg_data.mat'
env_attend_path = '/content/drive/MyDrive/Colab Notebooks/P&D_phase2/data/modified data/env1_data.mat'
env_unattend_path = '/content/drive/MyDrive/Colab Notebooks/P&D_phase2/data/modified data/env2_data.mat'

In [7]:
from scipy.io import loadmat

eeg  = loadmat(eeg_path);
env1 = loadmat(env_attend_path)
env2 = loadmat(env_unattend_path)
eeg = eeg['eeg_data']
env1 = env1['env1_data']
env2 = env2['env2_data']

In [8]:
print(eeg.shape)
print(env1.shape)
print(env2.shape)

(8700, 640, 64)
(8700, 640, 1)
(8700, 640, 1)


In [8]:
p = 5
labels = np.zeros((1740*p,1))
eeg,env1,env2, labels = batch_equalizer(eeg, env1, env2, labels)


In [10]:
print(eeg.shape)
print(env1.shape)
print(env2.shape)
print(labels.shape)

(17400, 640, 64)
(17400, 640, 1)
(17400, 640, 1)
(17400, 1)


In [9]:
model.compile(
      optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
      metrics=["acc"],
      loss=["binary_crossentropy"])

history = model.fit([eeg, env1, env2], labels,
          epochs=1000,validation_split=0.2,shuffle=True,
          verbose=2)
          

Epoch 1/1000
435/435 - 24s - loss: 0.6748 - acc: 0.6244 - val_loss: 0.8382 - val_acc: 0.0000e+00 - 24s/epoch - 55ms/step
Epoch 2/1000
435/435 - 8s - loss: 0.6643 - acc: 0.6250 - val_loss: 0.9079 - val_acc: 0.0000e+00 - 8s/epoch - 18ms/step
Epoch 3/1000
435/435 - 8s - loss: 0.6633 - acc: 0.6250 - val_loss: 1.0859 - val_acc: 0.0000e+00 - 8s/epoch - 18ms/step
Epoch 4/1000
435/435 - 8s - loss: 0.6639 - acc: 0.6250 - val_loss: 1.1460 - val_acc: 0.0000e+00 - 8s/epoch - 18ms/step
Epoch 5/1000
435/435 - 8s - loss: 0.6636 - acc: 0.6250 - val_loss: 1.0352 - val_acc: 0.0000e+00 - 8s/epoch - 18ms/step
Epoch 6/1000
435/435 - 8s - loss: 0.6634 - acc: 0.6250 - val_loss: 0.9596 - val_acc: 0.0000e+00 - 8s/epoch - 18ms/step
Epoch 7/1000
435/435 - 8s - loss: 0.6635 - acc: 0.6250 - val_loss: 0.9764 - val_acc: 0.0000e+00 - 8s/epoch - 18ms/step
Epoch 8/1000
435/435 - 8s - loss: 0.6631 - acc: 0.6250 - val_loss: 1.1148 - val_acc: 0.0000e+00 - 8s/epoch - 18ms/step
Epoch 9/1000
435/435 - 8s - loss: 0.6637 - acc

KeyboardInterrupt: ignored

In [ ]:
print(history.history.keys())

#plt.subplot(1,2,1)        
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('loss')
plt.ylabel('loss')  
plt.xlabel('epoch')
plt.legend(['train', 'valid'], loc='upper left')
plt.show()
